In [0]:
df = spark.read\
    .format("parquet")\
    .option("header", True)\
    .option("inferSchema", True)\
    .option("compression", "gzip")\
    .load("dbfs:/mnt/Anac/Silver/anac_silver.parquet")

In [0]:
colunas_relevantes = [
    'Classificacao_da_Ocorrência',
    'Danos_a_Aeronave',
    'Descricao_do_Tipo',
    'Fase_da_Operacao',
    'Historico',
    'UF',
    "Lesoes_Desconhecidas_Passageiros",
    "Lesoes_Desconhecidas_Terceiros",
    "Lesoes_Desconhecidas_Tripulantes",
    "Lesoes_Fatais_Passageiros",
    "Lesoes_Fatais_Terceiros",
    "Lesoes_Fatais_Tripulantes",
    "Lesoes_Graves_Passageiros",
    "Lesoes_Graves_Terceiros",
    "Lesoes_Graves_Tripulantes",
    "Lesoes_Leves_Passageiros",
    "Lesoes_Leves_Terceiros",
    "Lesoes_Leves_Tripulantes",
]
gold_df = df.select(colunas_relevantes)

In [0]:
colunas_lesoes = [
    "Lesoes_Desconhecidas_Passageiros",
    "Lesoes_Desconhecidas_Terceiros",
    "Lesoes_Desconhecidas_Tripulantes",
    "Lesoes_Fatais_Passageiros",
    "Lesoes_Fatais_Terceiros",
    "Lesoes_Fatais_Tripulantes",
    "Lesoes_Graves_Passageiros",
    "Lesoes_Graves_Terceiros",
    "Lesoes_Graves_Tripulantes",
    "Lesoes_Leves_Passageiros",
    "Lesoes_Leves_Terceiros",
    "Lesoes_Leves_Tripulantes",
]
gold_df = gold_df\
    .withColumn("Total de Lesões", sum(gold_df[qtd_lesoes] for qtd_lesoes in colunas_lesoes))

In [0]:
gold_df = gold_df.drop(*colunas_lesoes)

In [0]:
gold_df = gold_df\
    .withColumnRenamed('Classificacao_da_Ocorrência', 'Classificação da Ocorrência')\
    .withColumnRenamed('Danos_a_Aeronave', 'Danos a Aeronave')\
    .withColumnRenamed('Descricao_do_Tipo', 'Descrição do Tipo')\
    .withColumnRenamed('Fase_da_Operacao', 'Fase da Operação')\
    .withColumnRenamed('Historico', 'Histórico')\
    .withColumnRenamed('UF', 'Estado')

In [0]:
gold_df = gold_df\
    .where(gold_df['Classificação da Ocorrência'] != 'Indeterminado')\
    .where(gold_df['Danos a Aeronave'] != 'Desconhecida')\
    .where(gold_df['Danos a Aeronave'] != 'Sem Registro')\
    .where(gold_df['Descrição do Tipo'] != 'Sem Registro')\
    .where(gold_df['Fase da Operação'] != 'Indeterminada')\
    .where(gold_df['Fase da Operação'] != 'Sem Registro')\
    .where(gold_df['Histórico'] != 'Sem Registro')\
    .where(gold_df.Estado != 'Sem Registro')\
    .where(gold_df.Estado != 'Indeterminado')\
    .where(gold_df.Estado != 'Exterior')

In [0]:
from pyspark.sql.functions import current_timestamp, date_format, from_utc_timestamp

gold_df = gold_df.withColumn("Atualizado em", 
    date_format(
        from_utc_timestamp(
            current_timestamp(),
            "America/Sao_Paulo"
        ), 
        "yyyy-MM-dd HH:mm:ss" 
    )
) 

In [0]:
gold_df.write\
    .mode("overwrite")\
    .format("parquet")\
    .option("header", True)\
    .option("inferSchema", True)\
    .partitionBy("Estado")\
    .save("dbfs:/mnt/Anac/Gold/anac_gold_particionado")